# Planning Notebook

In [6]:
import math
import unittest
import numpy as np
from itertools import product
import tqdm
from tqdm import tqdm_notebook
import copy
import pickle
import time

# import gtsam
import gtsam
from gtsam import *
from gtsam.utils.test_case import GtsamTestCase

# import gtbook
import gtbook
from gtbook.display import *
from gtbook.discrete import *

# import local package
import gtsam_planner
from gtsam_planner import *

# import parser
import SASParser
from SASParser import SAS, Operator
from typing import List, Tuple, Callable, Dict, Iterable

import SasToGtsam
from SasToGtsam import SASToGTSAM

DiscreteKey = Tuple[int, int]

variables = Variables()
def pretty(obj): 
    return gtbook.display.pretty(obj, variables)

import graphviz
class show(graphviz.Source):
    """ Display an object with a dot method as a graph."""

    def __init__(self, obj):
        """Construct from object with 'dot' method."""
        # This small class takes an object, calls its dot function, and uses the
        # resulting string to initialize a graphviz.Source instance. This in turn
        # has a _repr_mimebundle_ method, which then renders it in the notebook.
        super().__init__(obj.dot())

In [7]:
sas = SAS()
sas_dir = "sas/gripper_example.sas"
sas.read_file(sas_dir)
converter = SASToGTSAM(sas)

In [8]:
# def plan(plan_length):
#     for k in range(2, plan_length):
#         print(k)
#         states = []
#         mutex_factors = []
#         op_factors = []

#         for i in range(k):
#             # generate state
#             state_t = converter.generate_state(i)
#             states.append(state_t)
#             # generate mutex factor for the state
#             mutex_factor_t = converter.generate_mutex_factor(state_t)
#             mutex_factors.append(mutex_factor_t)
#         for j in range(len(states)-1):
#             op_key = converter.generate_operator_key(j)
#             op_factor = converter.generate_op_factor(states[j], states[j+1], op_key)
#             op_factors.append(op_factor)
#         initial_factor = converter.generate_initial_factor(states[0])
#         goal_factor = converter.generate_goal_factor(states[-1])

#         graph = gtsam.DiscreteFactorGraph()
#         for m_factor in mutex_factors:
#             for f in m_factor:
#                 graph.push_back(f)

#         graph.push_back(goal_factor)
#         graph.push_back(initial_factor)

#         for op_factor in op_factors:
#             graph.push_back(op_factor)

#         val = graph.optimize()
#         if graph(val) == 0.0:
#             del graph
#             continue
#         else:
#             return graph, val, k
#     return "longer plan length?"

In [ ]:
start = time.time()
k = 2
states = []
mutex_factors = []
op_factors = []
frame_factors = []
for i in range(k):
    # generate state
    state_t = converter.generate_state(i)
    states.append(state_t)
    # generate mutex factor for the state
    mutex_factor_t = converter.generate_mutex_factor(state_t)
    mutex_factors.append(mutex_factor_t)

operators = []
for j in range(k-1):
    op_key = converter.generate_operator_key(j)
    operators.append(op_key)
    op_factor, frame_factor = converter.generate_frame_op_factor(states[j], states[j+1], op_key)
    op_factors.append(op_factor)
    frame_factors.append(frame_factor)
initial_factor = converter.generate_initial_factor(states[0])
goal_factor = converter.generate_goal_factor(states[-1])

graph = gtsam.DiscreteFactorGraph()
for m_factor in mutex_factors:
    for f in m_factor:
        graph.push_back(f)

graph.push_back(goal_factor)
graph.push_back(initial_factor)

for op_factor in op_factors:
    graph.push_back(op_factor)

for frame_factor in frame_factors:
    graph.push_back(frame_factor)

# this is where the planning happens
# enum OrderingType { COLAMD, METIS, NATURAL, CUSTOM };
# orderingType  = gtsam.Ordering.OrderingType.METIS
# dag = graph.maxProduct(orderingType)


ordering = gtsam.Ordering()
# initial to goal
for j in range(k):
    for state in states[j]:
        ordering.push_back(state[0])
    if j < k-1:
        ordering.push_back(operators[j][0])

# goal to initial
# for j in reversed(range(k)):
#     if j < k-1:
#         ordering.push_back(operators[j][0])
#     for state in states[j]:
#         ordering.push_back(state[0])

for j in reversed(range(k)):
    if j < k-1:
        ordering.push_back(operators[j][0])
    
for j in reversed(range(k)):
    for state in states[j]:
        ordering.push_back(state[0])

minigraph = gtsam.DiscreteFactorGraph()

dag = graph.maxProduct(ordering)
result = dag.argmax()

# result = graph.optimize()

end = time.time()
assert graph(result) == 1
print("time taken:", end - start)

In [4]:
start = time.time()
k = 2
states = []
mutex_factors = []
op_factors = []
frame_factors = []
for i in range(k):
    # generate state
    state_t = converter.generate_state(i)
    states.append(state_t)
    # generate mutex factor for the state
    mutex_factor_t = converter.generate_mutex_factor(state_t)
    mutex_factors.append(mutex_factor_t)

operators = []
for j in range(k-1):
    op_key = converter.generate_operator_key(j)
    operators.append(op_key)
    op_factor, frame_factor = converter.generate_frame_op_factor(states[j], states[j+1], op_key)
    op_factors.append(op_factor)
    frame_factors.append(frame_factor)
initial_factor = converter.generate_initial_factor(states[0])
goal_factor = converter.generate_goal_factor(states[-1])

graph = gtsam.DiscreteFactorGraph()
for m_factor in mutex_factors:
    for f in m_factor:
        graph.push_back(f)

graph.push_back(goal_factor)
graph.push_back(initial_factor)

for op_factor in op_factors:
    graph.push_back(op_factor)

for frame_factor in frame_factors:
    graph.push_back(frame_factor)

# this is where the planning happens
# enum OrderingType { COLAMD, METIS, NATURAL, CUSTOM };
# orderingType  = gtsam.Ordering.OrderingType.METIS
# dag = graph.maxProduct(orderingType)


ordering = gtsam.Ordering()
# for j in range(k):
#     for state in states[j]:
#         ordering.push_back(state[0])
#     if j < k-1:
#         ordering.push_back(operators[j][0])

# for j in reversed(range(k)):
#     if j < k-1:
#         ordering.push_back(operators[j][0])
#     for state in states[j]:
#         ordering.push_back(state[0])

for j in reversed(range(k)):
    if j < k-1:
        ordering.push_back(operators[j][0])
    
for j in reversed(range(k)):
    for state in states[j]:
        ordering.push_back(state[0])

minigraph = gtsam.DiscreteFactorGraph()

dag = graph.maxProduct(ordering)
result = dag.argmax()

# result = graph.optimize()

end = time.time()
assert graph(result) == 1
print("time taken:", end - start)

AssertionError: 

1. Divide the operator factor into two such that the first would evaluate only the precondition, second wouldonly evaluate the effect.

2. Change the ordering so that it would start from both initial state and the goal state and meet in the middle.

3. Change ordering so that we would evaluate the operators first and the states (we did this already)

4. Create factor that can just return true if there exists an action that gets from state_t to state_t+1

5. learned a bunch about elimination (hopefulyl)


In [6]:
converter.variables

Variable,Domain
0_0,"Atom at-robby(rooma), Atom at-robby(roomb)"
1_0,"Atom carry(ball1, left), Atom carry(ball2, left), Atom carry(ball3, left), Atom carry(ball4, left), Atom free(left)"
2_0,"Atom carry(ball1, right), Atom carry(ball2, right), Atom carry(ball3, right), Atom carry(ball4, right), Atom free(right)"
3_0,"Atom at(ball1, rooma), Atom at(ball1, roomb),"
4_0,"Atom at(ball2, rooma), Atom at(ball2, roomb),"
5_0,"Atom at(ball3, rooma), Atom at(ball3, roomb),"
6_0,"Atom at(ball4, rooma), Atom at(ball4, roomb),"
0_1,"Atom at-robby(rooma), Atom at-robby(roomb)"
1_1,"Atom carry(ball1, left), Atom carry(ball2, left), Atom carry(ball3, left), Atom carry(ball4, left), Atom free(left)"
2_1,"Atom carry(ball1, right), Atom carry(ball2, right), Atom carry(ball3, right), Atom carry(ball4, right), Atom free(right)"


In [ ]:
print(states)

[[(0, 2), (1, 5), (2, 5), (3, 3), (4, 3), (5, 3), (6, 3)], [(7, 2), (8, 5), (9, 5), (10, 3), (11, 3), (12, 3), (13, 3)], [(14, 2), (15, 5), (16, 5), (17, 3), (18, 3), (19, 3), (20, 3)], [(21, 2), (22, 5), (23, 5), (24, 3), (25, 3), (26, 3), (27, 3)], [(28, 2), (29, 5), (30, 5), (31, 3), (32, 3), (33, 3), (34, 3)], [(35, 2), (36, 5), (37, 5), (38, 3), (39, 3), (40, 3), (41, 3)], [(42, 2), (43, 5), (44, 5), (45, 3), (46, 3), (47, 3), (48, 3)], [(49, 2), (50, 5), (51, 5), (52, 3), (53, 3), (54, 3), (55, 3)], [(56, 2), (57, 5), (58, 5), (59, 3), (60, 3), (61, 3), (62, 3)], [(63, 2), (64, 5), (65, 5), (66, 3), (67, 3), (68, 3), (69, 3)], [(70, 2), (71, 5), (72, 5), (73, 3), (74, 3), (75, 3), (76, 3)], [(77, 2), (78, 5), (79, 5), (80, 3), (81, 3), (82, 3), (83, 3)]]


In [ ]:
for k in range(12):
    print(converter.variables[f"{0}_{k}"])

TypeError: 'Variables' object is not subscriptable

In [9]:
# converter.variables._variables

In [ ]:
print(graph.keys())

NameError: name 'graph' is not defined

In [ ]:
help(graph.optimize)

Help on method optimize in module gtsam.gtsam:

optimize(...) method of gtsam.gtsam.DiscreteFactorGraph instance
    optimize(self: gtsam.gtsam.DiscreteFactorGraph) -> gtsam::DiscreteValues



In [ ]:
print(graph)


size: 72
factor 0: MutexConstraint on 3 3 1 2 
factor 1: MutexConstraint on 4 4 1 2 
factor 2: MutexConstraint on 5 5 1 2 
factor 3: MutexConstraint on 6 6 1 2 
factor 4: MutexConstraint on 10 10 8 9 
factor 5: MutexConstraint on 11 11 8 9 
factor 6: MutexConstraint on 12 12 8 9 
factor 7: MutexConstraint on 13 13 8 9 
factor 8: MutexConstraint on 17 17 15 16 
factor 9: MutexConstraint on 18 18 15 16 
factor 10: MutexConstraint on 19 19 15 16 
factor 11: MutexConstraint on 20 20 15 16 
factor 12: MutexConstraint on 24 24 22 23 
factor 13: MutexConstraint on 25 25 22 23 
factor 14: MutexConstraint on 26 26 22 23 
factor 15: MutexConstraint on 27 27 22 23 
factor 16: MutexConstraint on 31 31 29 30 
factor 17: MutexConstraint on 32 32 29 30 
factor 18: MutexConstraint on 33 33 29 30 
factor 19: MutexConstraint on 34 34 29 30 
factor 20: MutexConstraint on 38 38 36 37 
factor 21: MutexConstraint on 39 39 36 37 
factor 22: MutexConstraint on 40 40 36 37 
factor 23: MutexConstraint on 41 41

In [ ]:
graph(val)

1.0

In [ ]:
op_consts = []
for i in range(graph.size()-1, graph.size()-k, -1):
    op_consts.append(graph.at(i))
val_list = []
for op_const in reversed(op_consts):
    print(converter.ops_names[val[op_const.operatorKey()]])

pick ball1 rooma left
pick ball2 rooma right
move rooma roomb
drop ball2 roomb right
drop ball1 roomb left
move roomb rooma
pick ball3 rooma left
pick ball4 rooma right
move rooma roomb
drop ball4 roomb right
drop ball3 roomb left


In [ ]:
# graph, val, k = plan(12)